In [1]:
import Pkg
Pkg.activate("../../tensor")
import tensor as ten
using ITensors

  Activating project at `c:\Users\kakin\home\workspace\tensor`


In [2]:
import LinearAlgebra as LA

In [3]:
#Index
#大きな足
α = Index(2, "α")
ξ = Index(2, "ξ")
β = Index(2, "β")
η = Index(2, "η")

#小さな足
i = Index(2, "i")
j = Index(2, "j")
k = Index(2, "k")
l = Index(2, "l")

#つぶす足
c1 = Index(2 , "c1")
c2 = Index(2 , "c2")

#定数K
K_const=-0.1

-0.1

In [4]:
#ここがおかしい
#4脚テンソルの定義
W = ten.four_leg_tensor_def(i, j, k, l,K_const)
#2脚テンソルの定義
C = ten.two_leg_tensor_def(α, β, c1, c2, K_const)
#3脚テンソルの定義
PR = ten.three_leg_tensor_def(α, ξ, l, c1, K_const)
#3脚テンソルの定義
PC = ten.three_leg_tensor_def(β, η , k, c1, K_const)

ITensor ord=3 (dim=2|id=814|"β") (dim=2|id=680|"η") (dim=2|id=201|"k")
NDTensors.Dense{Float64, Vector{Float64}}

In [5]:
C2=PR*C*PC*W

ITensor ord=4 (dim=2|id=560|"ξ") (dim=2|id=680|"η") (dim=2|id=547|"i") (dim=2|id=43|"j")
NDTensors.Dense{Float64, Vector{Float64}}

In [6]:
ξi = combiner(ξ, i)
ηj = combiner(η, j)

C2_ext = ξi * C2 * ηj

α = Index(dim(uniqueinds(ξi)[1]), "α")
β = Index(dim(uniqueinds(ηj)[1]), "β")
replaceinds!(C2_ext, uniqueinds(ξi)[1] => α)
replaceinds!(C2_ext, uniqueinds(ηj)[1] => β)

C2=C2_ext

ITensor ord=2 (dim=4|id=445|"α") (dim=4|id=433|"β")
NDTensors.Dense{Float64, Vector{Float64}}

In [7]:
# 固有値分解
D, U = eigen(C2, α, β)

# インデックスの取り出し
dl = uniqueind(D, U)
dr = commonind(D, U)

# 正しい方法でUlを作成（置き換えは個別に！）
Ul = replaceinds(U, (β => α, dr => dl))

# 誤差チェック
diff = norm(C2 * U - Ul * D)
println("norm(C2 * U - Ul * D) = ", diff)
println("一致する？ ", diff < 1e-11)



norm(C2 * U - Ul * D) = 1.1791567914286052e-12
一致する？ true


In [32]:
function Sort_Diagonal(D::ITensor)
    Dnew=copy(D)

    #対角要素の並び替え
    for i in 1:size(D)[1]
        for j in (i+1):size(D)[2]
            if abs(real(Dnew[i,i])) < abs(real(Dnew[j,j]))
                tmp=Dnew[i,i]
                Dnew[i,i]=Dnew[j,j]
                Dnew[j,j]=tmp
            end
        end
    end

    
    return Dnew
end

Dnew=Sort_Diagonal(D)
@show D

D = ITensor ord=2
Dim 1: (dim=4|id=217|"Link,eigen")'
Dim 2: (dim=4|id=217|"Link,eigen")
NDTensors.Diag{ComplexF64, Vector{ComplexF64}}
 4×4
 -2.2167027949896285 + 0.0im                 0.0 + 0.0im                  0.0 + 0.0im                 0.0 + 0.0im
                 0.0 + 0.0im  -111.7235422814734 + 0.0im                  0.0 + 0.0im                 0.0 + 0.0im
                 0.0 + 0.0im                 0.0 + 0.0im  0.22317659368484247 + 0.0im                 0.0 + 0.0im
                 0.0 + 0.0im                 0.0 + 0.0im                  0.0 + 0.0im  1109.6947235926991 + 0.0im


ITensor ord=2 (dim=4|id=217|"Link,eigen")' (dim=4|id=217|"Link,eigen")
NDTensors.Diag{ComplexF64, Vector{ComplexF64}}

In [33]:
function Sort_idx(D::ITensor,i::Index{Int64},j::Index{Int64})
    # 対角要素を取り出す
    diag_elements=Float64[]
    for i in 1:size(D)[1]
        push!(diag_elements,D[i,i])
    end
    
    # 絶対値でソートしたインデックスを取得
    sorted_indices = sortperm(abs.(diag_elements))
    
    return sorted_indices
end

idx=Sort_idx(D,i,j)
@show D
@show idx

D = ITensor ord=2
Dim 1: (dim=4|id=217|"Link,eigen")'
Dim 2: (dim=4|id=217|"Link,eigen")
NDTensors.Diag{ComplexF64, Vector{ComplexF64}}
 4×4
 -2.2167027949896285 + 0.0im                 0.0 + 0.0im                  0.0 + 0.0im                 0.0 + 0.0im
                 0.0 + 0.0im  -111.7235422814734 + 0.0im                  0.0 + 0.0im                 0.0 + 0.0im
                 0.0 + 0.0im                 0.0 + 0.0im  0.22317659368484247 + 0.0im                 0.0 + 0.0im
                 0.0 + 0.0im                 0.0 + 0.0im                  0.0 + 0.0im  1109.6947235926991 + 0.0im
idx = [3, 1, 2, 4]


4-element Vector{Int64}:
 3
 1
 2
 4

In [15]:
χ_num=3
χ=Index(χ_num,"χ")

(dim=3|id=158|"χ")

In [ ]:
function Restrict_Diagonal(D::ITensor, χ::Int64,a1::Index{Int64},a2::Index{Int64})
    if size(D)[1]<χ
        return D
    end

    resD=ITensor(a1',a2)

    for i in 1:χ
        resD[i,i]=D[i,i]
    end

    return resD
end
resD = Restrict_Diagonal(D, χ_num,χ,χ)
@show D
@show resD


D = ITensor ord=2
Dim 1: (dim=4|id=217|"Link,eigen")'
Dim 2: (dim=4|id=217|"Link,eigen")
NDTensors.Diag{ComplexF64, Vector{ComplexF64}}
 4×4
 -2.2167027949896285 + 0.0im                 0.0 + 0.0im                  0.0 + 0.0im                 0.0 + 0.0im
                 0.0 + 0.0im  -111.7235422814734 + 0.0im                  0.0 + 0.0im                 0.0 + 0.0im
                 0.0 + 0.0im                 0.0 + 0.0im  0.22317659368484247 + 0.0im                 0.0 + 0.0im
                 0.0 + 0.0im                 0.0 + 0.0im                  0.0 + 0.0im  1109.6947235926991 + 0.0im
resD = ITensor ord=2
Dim 1: (dim=3|id=158|"χ")'
Dim 2: (dim=3|id=158|"χ")
NDTensors.Dense{ComplexF64, Vector{ComplexF64}}
 3×3
 -2.2167027949896285 + 0.0im                 0.0 + 0.0im                  0.0 + 0.0im
                 0.0 + 0.0im  -111.7235422814734 + 0.0im                  0.0 + 0.0im
                 0.0 + 0.0im                 0.0 + 0.0im  0.22317659368484247 + 0.0im


ITensor ord=2 (dim=3|id=158|"χ")' (dim=3|id=158|"χ")
NDTensors.Dense{ComplexF64, Vector{ComplexF64}}

In [42]:
function Restrict_EigenvecsU(U::ITensor, χ::Int64,a1::Index{Int64},a2::Index{Int64},idx::Vector{Int64})
    if size(U)[1]<χ
        return U
    end

    resU=ITensor(a1,a2)

    #並び替え
    newU=copy(U)
    
    for i in 1:size(U)[1]
        cou=1
        for j in idx
            newU[i,cou]=U[i,j]
            cou+=1
        end
    end
    

    #大きさを制限
    for i in 1:size(U)[1]
        for j in 1:χ
            resU[i,j]=newU[i,j]
        end
    end

    return resU
end
resU = Restrict_EigenvecsU(U, χ_num,β,χ,idx)
@show U
@show resU

U = ITensor ord=2
Dim 1: (dim=4|id=433|"β")
Dim 2: (dim=4|id=217|"Link,eigen")
NDTensors.Dense{ComplexF64, Vector{ComplexF64}}
 4×4
  0.5101295572748251 + 0.0im  -0.4896609386040289 + 0.0im  0.49490655590318317 + 0.0im   0.5050420783697566 + 0.0im
 0.48966093860404125 + 0.0im   0.5101295572748119 + 0.0im  -0.5050420783697509 + 0.0im    0.494906555903189 + 0.0im
 -0.4896609386040415 + 0.0im  -0.5101295572748379 + 0.0im   -0.505042078369763 + 0.0im  0.49490655590318916 + 0.0im
 -0.5101295572748253 + 0.0im   0.4896609386040544 + 0.0im  0.49490655590319454 + 0.0im    0.505042078369757 + 0.0im
resU = ITensor ord=2
Dim 1: (dim=4|id=433|"β")
Dim 2: (dim=3|id=158|"χ")
NDTensors.Dense{ComplexF64, Vector{ComplexF64}}
 4×3
 0.49490655590318317 + 0.0im   0.5101295572748251 + 0.0im  -0.4896609386040289 + 0.0im
 -0.5050420783697509 + 0.0im  0.48966093860404125 + 0.0im   0.5101295572748119 + 0.0im
  -0.505042078369763 + 0.0im  -0.4896609386040415 + 0.0im  -0.5101295572748379 + 0.0im
 0.49490655590319

ITensor ord=2 (dim=4|id=433|"β") (dim=3|id=158|"χ")
NDTensors.Dense{ComplexF64, Vector{ComplexF64}}

In [43]:
function Restrict_EigenvecsUl(Ul::ITensor, χ::Int64,a1::Index,a2::Index)
    if size(Ul)[1]<χ
        return Ul
    end

    resUl=ITensor(a2',a1)

    #並び替え
    newUl=copy(Ul)
    
    for i in 1:size(Ul)[1]
        cou=1
        for j in idx
            newUl[i,cou]=Ul[i,j]
            cou+=1
        end
    end

    for i in 1:χ
        for j in 1:size(Ul)[2]
            resUl[i,j]=newUl[i,j]
        end
    end

    return resUl
end
resUl = Restrict_EigenvecsUl(Ul, χ_num,α,χ)
@show Ul
@show resUl

Ul = ITensor ord=2
Dim 1: (dim=4|id=445|"α")
Dim 2: (dim=4|id=217|"Link,eigen")'
NDTensors.Dense{ComplexF64, Vector{ComplexF64}}
 4×4
  0.5101295572748251 + 0.0im  -0.4896609386040289 + 0.0im  0.49490655590318317 + 0.0im   0.5050420783697566 + 0.0im
 0.48966093860404125 + 0.0im   0.5101295572748119 + 0.0im  -0.5050420783697509 + 0.0im    0.494906555903189 + 0.0im
 -0.4896609386040415 + 0.0im  -0.5101295572748379 + 0.0im   -0.505042078369763 + 0.0im  0.49490655590318916 + 0.0im
 -0.5101295572748253 + 0.0im   0.4896609386040544 + 0.0im  0.49490655590319454 + 0.0im    0.505042078369757 + 0.0im
resUl = ITensor ord=2
Dim 1: (dim=3|id=158|"χ")'
Dim 2: (dim=4|id=445|"α")
NDTensors.Dense{ComplexF64, Vector{ComplexF64}}
 3×4
 0.49490655590318317 + 0.0im   0.5101295572748251 + 0.0im  -0.4896609386040289 + 0.0im   0.5050420783697566 + 0.0im
 -0.5050420783697509 + 0.0im  0.48966093860404125 + 0.0im   0.5101295572748119 + 0.0im    0.494906555903189 + 0.0im
  -0.505042078369763 + 0.0im  -0.489660938

ITensor ord=2 (dim=3|id=158|"χ")' (dim=4|id=445|"α")
NDTensors.Dense{ComplexF64, Vector{ComplexF64}}

In [19]:
Ul*D*U

ITensor ord=2 (dim=4|id=445|"α") (dim=4|id=433|"β")
NDTensors.Dense{ComplexF64, Vector{ComplexF64}}

In [44]:
resUl

ITensor ord=2 (dim=3|id=158|"χ")' (dim=4|id=445|"α")
NDTensors.Dense{ComplexF64, Vector{ComplexF64}}

In [21]:
resD

ITensor ord=2 (dim=3|id=158|"χ")' (dim=3|id=158|"χ")
NDTensors.Dense{ComplexF64, Vector{ComplexF64}}

In [22]:
size(resU)|>display
size(resUl)|>display
size(resD)|>display

(4, 3)

(3, 4)

(3, 3)

In [23]:
A=resUl*resD*resU
#テスト

ITensor ord=2 (dim=4|id=445|"α") (dim=4|id=433|"β")
NDTensors.Dense{ComplexF64, Vector{ComplexF64}}

In [24]:
C2

ITensor ord=2 (dim=4|id=445|"α") (dim=4|id=433|"β")
NDTensors.Dense{Float64, Vector{Float64}}

In [25]:
α

(dim=4|id=445|"α")

In [26]:
PR

ITensor ord=3 (dim=2|id=885|"α") (dim=2|id=560|"ξ") (dim=2|id=610|"l")
NDTensors.Dense{Float64, Vector{Float64}}

In [27]:
PR2=PR*W

ITensor ord=5 (dim=2|id=885|"α") (dim=2|id=560|"ξ") (dim=2|id=547|"i") (dim=2|id=43|"j") (dim=2|id=201|"k")
NDTensors.Dense{Float64, Vector{Float64}}

In [28]:
αk = combiner(α, k)
ξi = combiner(ξ, i)

PR2_ext = ξi * PR2 * αk

#=
α = Index(dim(uniqueinds(ξi)[1]), "α")
ρ = Index(dim(uniqueinds(αk)[1]), "ρ")
replaceinds!(PR2_ext, uniqueinds(ξi)[1] => α)
replaceinds!(PR2_ext, uniqueinds(αk)[1] => ρ)

PR2=PR2_ext
=#
#=
ξ = Index(dim(uniqueinds(ξi)[1]), "ξ")
α = Index(dim(uniqueinds(αk)[1]), "α")
replaceinds!(PR2_ext, uniqueinds(ξi)[1] => ξ)
replaceinds!(PR2_ext, uniqueinds(βj)[1] => α)

PR2=PR2_ext
=#

ErrorException: Trying to contract a tensor with indices:

((dim=4|id=610|"CMB,Link"), (dim=2|id=885|"α"), (dim=2|id=43|"j"), (dim=2|id=201|"k"))

and labels:

(2, 3, 4, -1)

with a combiner tensor with indices:

((dim=8|id=989|"CMB,Link"), (dim=4|id=445|"α"), (dim=2|id=201|"k"))

and labels:

(5, 6, -1).

This is not a valid combiner contraction.

If you are combining, the combined index of the combiner should be the only one uncontracted.

If you are uncombining, the combined index of the combiner should be the only one contracted.

By convention, the combined index should be the index in position 1 of the combiner tensor.


In [29]:
#=using ITensors

# 対角化（例: A is ITensor with indices α, β）
D, U = eigen(C2, α, β)

dr = commonind(D, U)
dl = uniqueind(D, U)

χ = 2

# ✅ 新しい次元χのIndexを新規生成（タグは同じでもOK）
drχ = Index(χ, "Link,eigen")
dlχ = prime(drχ)

# ✅ Uのインデックス置き換え（単一の置換ならこれで十分）
Uχ = replaceind(U, dr => drχ)

# ✅ Dχ の構築
Dχ = ITensor(dlχ, drχ)
for n in 1:χ
    Dχ[dlχ => n, drχ => n] = D[dl => n, dr => n]
end

# ✅ Ulχ の構成（ここも replaceind で十分）
Ulχ = replaceind(Uχ, β => α)
Ulχ = replaceind(Ulχ, drχ => dlχ)

# ✅ 再構成して誤差確認
Aχ = Ulχ * Dχ
println("圧縮誤差: ", norm(C2 - Aχ))
=#

In [30]:
D
norm(Ul*D*U-C2)

154.8660575743989

In [31]:
A=Array(C2,α,β)
Base.print_matrix(stdout,A)

 253.49616979434984  249.95712926003972  304.66466954142385  312.7072764780771
 249.95712926003972  244.49265776061023  299.22179615334585  304.66466954142385
 304.6646695414238   299.22179615334585  244.4926577606102   249.95712926003972
 312.7072764780772   304.66466954142385  249.95712926003972  253.49616979434984